# News clustering : Exploring different clustering methods

Objective: The goal is to partition news articles from a specific timeframe (a 7-day sliding window) into semantically correlated groups, where each cluster ideally represents a real-world financial event.
- Algorithm: Use Agglomerative Clustering (hierarchical). This is a bottom-up approach where each article begins as its own cluster before being successively merged with others.
- Distance Metric: Use Cosine Distance, which is the most suitable metric for comparing high-dimensional text vectors.
- Linkage Criterion: Use Average Linkage, which minimizes the average distance between all observations of pairs of clusters.
- Determining $k$ (Number of Clusters): Since the number of events fluctuates daily, the optimal $k$ is found using the Silhouette Maximization method.
    - Iterate the algorithm for $k$ values ranging from 2 to 10.
    - Calculate the average Silhouette Coefficient for each $k$.
    - Select the value of $k$ that yields the highest score.
- Centroid Calculation: Once clusters are formed, compute a centroid for each group using the median of the cluster embeddings, as it is more robust to noise than the mean.

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import AgglomerativeClustering, KMeans
from pyclustering.cluster.kmedians import kmedians
# from sklearn.metrics import silhouette_score
# from scipy.spatial.distance import cosine
import os
import sys
import re

sys.path.append(os.path.abspath(os.path.join('..')))
from src.news_clustering import *

### Data preparation

In [2]:
news_features = pd.read_csv('../data/for_models/news_features.csv')
news_features['date'] = pd.to_datetime(news_features['date'])

In [3]:
# Converting String to Array
def string_to_array(s):
    s = re.sub(r'[\[\]\n]', '', s)
    # String to Numpy with type float
    return np.fromstring(s, sep=' ')

# We transform 'embedding' column into a stack of numpy arrays
X = np.stack(news_features['embedding'].apply(string_to_array).values)

### Choice of k clusters

In [4]:
# Exemple : Choice of clusters for the week before the SVB collapse
START_DATE = pd.to_datetime('2023-03-03')
END_DATE = pd.to_datetime('2023-03-09') # Exemple

# We create a mask to filter news_features for the chosen week
mask_week = (news_features['date'] >= START_DATE) & (news_features['date'] <= END_DATE)

# Evaluation
res = run_clustering_evaluation(X[mask_week], k_range=range(2, 11), min_samples=2)
print(res)

    k  Agglomerative   K-Means  K-Medians
0   2       0.368430  0.167319   0.071335
1   3       0.309424  0.106892   0.103260
2   4       0.276423  0.132134   0.089046
3   5       0.308400  0.141474   0.047022
4   6       0.309295  0.147214   0.092736
5   7       0.312366  0.148055   0.098489
6   8       0.305844  0.154694   0.145416
7   9       0.358599  0.145295   0.148018
8  10            NaN  0.154351   0.127875


In [5]:
plot_clustering_comparison(res)

So, the adequate number of clusters for the week before the SVB collapse seems to be 2 or 9, depending on the minimal number of news in each cluster.

In [6]:
fig = visualize_hac_tsne_range(X, news_features, start_date=START_DATE, end_date=END_DATE, k=2, perplexity=10, min_samples=2)
fig.show()

As we could expect, this week (precedind the day of collapse) seems to have a strong bipolarity. The news are essentially about the incoming event.

In [13]:
# Stable HAC
Z_stable, headlines, X_stable = compute_stable_hac_linkage(
    X, news_features, "2023-03-03", "2023-03-09", k=2, min_samples=2
)

# Dendogram
fig = plot_hac_dendrogram_plotly(Z_stable, headlines, "2023-03-03", "2023-03-09")
fig.show()

### Centroid computation for Hierarchical Clustering

Unlike K-Means, Agglomerative Clustering (HAC) does not inherently define cluster centers. However, for the subsequent Tweet Assignment phase, we must compare each tweet to the detected "events." The centroid serves as the numerical signature of the financial event.

The Carta et al. paper specifies using the median of the cluster vectors rather than the mean, as it is significantly more robust to outliers (noisy articles) that may remain within the group.

This will be developped in the newt step